In [1]:
import torch

# 1. Check if the Intel XPU backend is available
xpu_available = torch.xpu.is_available()
print(f"Intel GPU (XPU) available: {xpu_available}")

if xpu_available:
    # 2. Get the specific name of your Intel GPU
    device_name = torch.xpu.get_device_name(0)
    print(f"Device Name: {device_name}")
    
    # 3. Create a small tensor and move it to the Intel GPU
    device = torch.device("xpu")
    x = torch.randn(3, 3).to(device)
    
    # 4. Perform a small calculation to confirm it works
    y = x * x
    print("\nCalculation successful!")
    print(f"Tensor is currently on: {y.device}")
else:
    print("Intel GPU not detected. Ensure your Intel Graphics drivers are up to date.")


Intel GPU (XPU) available: False
Intel GPU not detected. Ensure your Intel Graphics drivers are up to date.


Now let's see YOLO OpenVINNO export
How it compares to regular cpu inference speed

In [18]:
from ultralytics import YOLO

# Load a pre-trained YOLO model
# model = YOLO('yolov8n.pt')
model = YOLO('yolo11n.pt')

# Export the model to OpenVINO format
# model.export(format='openvino') # creates 'yolo11n_openvino_model/'

# Load the exported OpenVINO model
ov_model = YOLO('yolo11n_openvino_model/')

# image_url = "https://c8.alamy.com/comp/P6YB2N/los-angeles-usa-june-29-unidentified-random-people-in-the-streets-of-downtown-of-los-angeles-ca-on-june-29-2018-P6YB2N.jpg"
image_url = "https://thumbs.dreamstime.com/b/cute-cat-sleeping-street-car-random-58655731.jpg"
# run inference on an image (original model)
print("Running inference with the original model...")
results = model.predict(source=image_url)

print("="*20)

# run inference on an image (openvino model)
print("Running inference with the OpenVINO model...")
results = ov_model.predict(source=image_url)

WARNING Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Running inference with the original model...

Found https://thumbs.dreamstime.com/b/cute-cat-sleeping-street-car-random-58655731.jpg locally at cute-cat-sleeping-street-car-random-58655731.jpg
image 1/1 C:\Users\ianwr\cute-cat-sleeping-street-car-random-58655731.jpg: 480x640 1 cat, 95.7ms
Speed: 4.6ms preprocess, 95.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Running inference with the OpenVINO model...
Loading yolo11n_openvino_model/ for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference on (CPU)...

Found https://thumbs.dreamstime.com/b/cute-cat-sleeping-street-car-random-58655731.jpg locally at cute-cat-sleeping-street-car-random-58655731.jpg
image 1/1 C:\Users\ianwr\cute-cat-sleeping-street-car-random-58655731.jpg: 640x640 1 car, 1 cat, 202.5ms
Speed: 3.8ms preprocess

In [1]:
import cv2
from ultralytics import YOLO

# ---- CONFIG ----
STREAM_URL = "http://192.168.0.15:4747/video" # coming from my DroidCAM app on Android (substitute for Live Camera)
MODEL_PATH = "yolo11n.pt"   # change to -seg or -pose if needed
CONF_THRESH = 0.5

# ---- LOAD MODEL ----
model = YOLO(MODEL_PATH)

# ---- OPEN STREAM ----
cap = cv2.VideoCapture(STREAM_URL)

if not cap.isOpened():
    raise RuntimeError("❌ Could not open DroidCAM stream")

# Optional: reduce latency
cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)

print("🦇 Bat-vision online. Press Q to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Run inference
    results = model(frame, conf=CONF_THRESH, verbose=False)

    # Draw results
    annotated = results[0].plot()

    cv2.imshow("YOLO DroidCAM", annotated)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


🦇 Bat-vision online. Press Q to quit.


: 

: 

: 